In [9]:
!pip install PyPDF2
!pip install PyMuPDF
!pip install datasets
!pip install transformers
!pip install torch_xla
!pip install cloud-tpu-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pydrive2 1.6.3 requires google-api-python-client>=1.12.5, but you have goo

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import fitz  # PyMuPDF


def extract_job_role(folder_name):
    # Extract job role from folder name
    return folder_name

def extract_section_text(pdf_path):
    # Initialize variables to store extracted details
    sections = {
        "Skills": "",
        "Education": ""
    }
    job_role = ""
    current_section = None

    # Extract text from PDF using PyMuPDF
    doc = fitz.open(pdf_path)
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text = page.get_text()

        # Split text into lines and process each line
        lines = text.split('\n')
        for line in lines:
            # Extract job role from the first line
            if not job_role:
                job_role = line.strip()
                continue

            line = line.strip()  # Remove leading and trailing whitespace

            # Check if the line contains a section header
            for section_name in sections.keys():
                if section_name in line:
                    current_section = section_name
                    continue

            # If in a section, add the line to the appropriate section
            if current_section and line != current_section:
                sections[current_section] += line + ' '

    # Close the PDF document
    doc.close()

    return job_role, sections

# Set the path to the folder containing job role subfolders
main_folder = '/content/drive/MyDrive/Capital Placements/archive-2/data/data'

# Create a list to store extracted details for each resume
all_resume_details = []

# Iterate through job role subfolders
for job_role_folder in os.listdir(main_folder):
    job_role_path = os.path.join(main_folder, job_role_folder)

    # Ensure it's a directory
    if os.path.isdir(job_role_path):
        print(f"Processing job role: {job_role_folder}")

        # Iterate through resumes in the job role folder
        for filename in os.listdir(job_role_path):
            if filename.endswith('.pdf'):
                pdf_path = os.path.join(job_role_path, filename)
                job_role, extracted_sections = extract_section_text(pdf_path)

                # Store the extracted details in a dictionary
                resume_details = {
                    "filename": filename,
                    "category": job_role,
                    "skills": extracted_sections["Skills"].strip(),
                    "education": extracted_sections["Education"].strip()
                }

                # Append the details to the list
                all_resume_details.append(resume_details)

                # Print the extracted text from each section
                print(f"File: {filename}")
                print(f"Category: {job_role}")
                for section_name, section_text in extracted_sections.items():
                    print(f"{section_name}:")
                    print(section_text.strip())  # Remove extra whitespace
                print("-" * 50)

# Now, you have all extracted details in the 'all_resume_details' list.
# You can use this list for matching job descriptions or any other further analysis.


Streaming output truncated to the last 5000 lines.
Bachelor of Science : Accounting , 1993 Rhode Island College - City , State Accounting
--------------------------------------------------
File: 87635012.pdf
Category: PRINCIPAL ACCOUNTANT
Skills:
Microsoft Excel, Peachtree, PeopleSoft, SAP, Sarbanes-Oxley, SQL
Education:
Master of Business Administration , Finance 2002 Rutgers University ï¼​ City , State Bachelor of Business Administration , Accounting 1991 Pace University ï¼​ City , State
--------------------------------------------------
File: 14224370.pdf
Category: ACCOUNTANT
Skills:
Technical Skills Accounting Software: PeopleSoft / QuickBooks Tax Software: Drake / Ultra Tax / TurboTax Operating Systems: XP Windows Professional / Windows 7/ Windows 8 Microsoft Office: Word / Advanced MS Excel skills /MS Access / PowerPoint / Outlook PDF: Adobe Reader / Adobe Acrobat X Experience 04/2012 to 06/2014 Accountant Company Name ï¼​ City , State Prepare federal tax returns for individuals 

In [3]:
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions")


# List available columns (replace 'dataset_name' with your actual dataset name)
print(dataset['train'].column_names)  # You might need to check 'test', 'validation', or other splits

# Access job descriptions using the correct key
job_descriptions = dataset['train']['job_description'][:15]
job_position= dataset['train']['position_title'][:15]


# Now, you can continue with the rest of the code to organize the data as shown in the previous response.


['company_name', 'job_description', 'position_title', 'description_length', 'model_response']


In [ ]:
job_data = []
for description in job_descriptions:
    job_data.append({
        'company_name': dataset['train']['company_name'][:15],
        'job_description': description,
        'position_title': dataset['train']['position_title'][:15],
        'description_length': len(description),
        'model_response': dataset['train']['model_response'][:15]
    })

In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

# Define a function for text preprocessing
def preprocess_text(text):
    # Remove special characters and convert to lowercase
    text = text.lower()
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Join the tokens back into text
    preprocessed_text = ' '.join(filtered_tokens)

    return preprocessed_text

# Apply preprocessing to each job description
preprocessed_job_descriptions = [preprocess_text(description) for description in job_descriptions]

for resume_details in all_resume_details:
    resume_details['skills'] = preprocess_text(resume_details['skills'])
    resume_details['education'] = preprocess_text(resume_details['education'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the DistilBERT model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name).to(device)
model.eval()  # Set the model to evaluation mode

# Initialize maximum sequence length
max_sequence_length = 512

# Tokenize job descriptions (you need to define job_descriptions)
job_description_tokens = [tokenizer.encode(description, add_special_tokens=True, max_length=max_sequence_length, truncation=True, return_tensors='pt').to(device) for description in job_descriptions]

# Initialize a dictionary to store the top 5 resumes for each job description
top_resumes = {job_description: [] for job_description in job_descriptions}

# Iterate through each resume details (you need to define all_resume_details)
for resume_details in all_resume_details:
    filename = resume_details['filename']
    category = resume_details['category']
    skills_text = resume_details['skills']
    education_text = resume_details['education']

    # Tokenize skills and education texts
    skills_tokens = tokenizer.encode(skills_text, add_special_tokens=True, max_length=max_sequence_length, truncation=True, return_tensors='pt').to(device)
    education_tokens = tokenizer.encode(education_text, add_special_tokens=True, max_length=max_sequence_length, truncation=True, return_tensors='pt').to(device)

    # Calculate embeddings for job descriptions, skills, and education
    job_description_embeddings = [model(tokens).last_hidden_state.mean(dim=1) for tokens in job_description_tokens]
    skills_embeddings = model(skills_tokens).last_hidden_state.mean(dim=1)
    education_embeddings = model(education_tokens).last_hidden_state.mean(dim=1)

    # Calculate cosine similarities for skills
    similarities_skills = [cosine_similarity(job_emb.detach().cpu().numpy(), skills_embeddings.detach().cpu().numpy())[0][0] for job_emb in job_description_embeddings]

    # Calculate cosine similarities for education
    similarities_education = [cosine_similarity(job_emb.detach().cpu().numpy(), education_embeddings.detach().cpu().numpy())[0][0] for job_emb in job_description_embeddings]

    # Combine similarities (you can use different strategies here)
    similarities_combined = [0.5 * sim_skills + 0.5 * sim_education for sim_skills, sim_education in zip(similarities_skills, similarities_education)]

    # Find the top 5 resumes with the highest similarity for each job description
    top_5_indices = sorted(range(len(similarities_combined)), key=lambda i: similarities_combined[i], reverse=True)[:5]

    # Store the top 5 resumes for each job description
    for description, index in zip(job_descriptions, top_5_indices):
        top_resumes[description].append({
            'filename': filename,
            'category': category,
            'cosine_similarity': similarities_combined[index]
        })
print(top_5_indices)
# Print or process the top 5 resumes for each job description as needed
for job_description, resumes in top_resumes.items():
    print(f"Job Description: {job_description}")
    print("Top 5 Resumes:")
    for resume in resumes:
        print(f"Resume: {resume['filename']}")
        print(f"Category: {resume['category']}")
        print(f"Cosine Similarity: {resume['cosine_similarity']}")
    print("\n")


Streaming output truncated to the last 5000 lines.
Resume: 78016758.pdf
Category: CONSULTANT
Cosine Similarity: 0.5962507724761963
Resume: 14517953.pdf
Category: CONSULTANT
Cosine Similarity: 0.5851131677627563
Resume: 95429627.pdf
Category: CONSULTANT
Cosine Similarity: 0.5481925904750824
Resume: 39308779.pdf
Category: CONSULTANT
Cosine Similarity: 0.5776297450065613
Resume: 88691367.pdf
Category: CONSULTANT
Cosine Similarity: 0.570840060710907
Resume: 51724595.pdf
Category: CONSULTANT
Cosine Similarity: 0.5892770290374756
Resume: 26234972.pdf
Category: MEDIA CONSULTANT
Cosine Similarity: 0.5966546535491943
Resume: 32433431.pdf
Category: CONSULTANT
Cosine Similarity: 0.5826499462127686
Resume: 27726066.pdf
Category: CONSULTANT
Cosine Similarity: 0.60042405128479
Resume: 65062795.pdf
Category: WMS CONSULTANT
Cosine Similarity: 0.60738205909729
Resume: 51432451.pdf
Category: CONSULTANT
Cosine Similarity: 0.5638171434402466
Resume: 27096471.pdf
Category: CONSULTANT
Cosine Similarity: 0.5

In [ ]:
import torch
from transformers import DistilBertModel, DistilBertTokenizer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the DistilBERT model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertModel.from_pretrained(model_name).to(device)
model.eval()  # Set the model to evaluation mode

# Initialize maximum sequence length
max_sequence_length = 512

# Create a dictionary to store resumes by category
resumes_by_category = {}

# Iterate through each resume details (you need to define all_resume_details)
for resume_details in all_resume_details:
    filename = resume_details['filename']
    category = resume_details['category']
    skills_text = resume_details['skills']
    education_text = resume_details['education']

    # Tokenize skills and education texts
    skills_tokens = tokenizer.encode(skills_text, add_special_tokens=True, max_length=max_sequence_length, truncation=True, return_tensors='pt').to(device)
    education_tokens = tokenizer.encode(education_text, add_special_tokens=True, max_length=max_sequence_length, truncation=True, return_tensors='pt').to(device)

    # Calculate embeddings for skills and education
    skills_embeddings = model(skills_tokens).last_hidden_state.mean(dim=1)
    education_embeddings = model(education_tokens).last_hidden_state.mean(dim=1)

    # Create a combined embedding for skills and education
    combined_embeddings = 0.5 * skills_embeddings + 0.5 * education_embeddings

    # Check if the category exists in the dictionary, if not, initialize it
    if category not in resumes_by_category:
        resumes_by_category[category] = []

    # Append the resume details to the category
    resumes_by_category[category].append({
        'filename': filename,
        'combined_embeddings': combined_embeddings
    })

# Create a dictionary to store the top 5 resumes for each job description
top_5_resumes_per_description = {}

# Iterate through each job description (you need to define job_descriptions)
for job_description in preprocessed_job_descriptions:
    # Tokenize the job description
    job_description_tokens = tokenizer.encode(job_description, add_special_tokens=True, max_length=max_sequence_length, truncation=True, return_tensors='pt').to(device)

    # Calculate embeddings for the job description
    job_description_embeddings = model(job_description_tokens).last_hidden_state.mean(dim=1)

    # Get the corresponding job_position from the job description dataset
    job_position = job_description_dataset[job_description]  # Replace with your actual code to retrieve job_position

    # Match the job_position with the category and calculate cosine similarity
    category = job_position  # Assuming job_position matches category
    similarities = []
    for resume_details in resumes_by_category.get(category, []):
        combined_embeddings = resume_details['combined_embeddings']
        similarity = cosine_similarity(job_description_embeddings.detach().cpu().numpy(), combined_embeddings.detach().cpu().numpy())[0][0]
        similarities.append((resume_details, similarity))

    # Sort resumes by cosine similarity
    sorted_resumes = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Take the top 5 resumes
    top_5_resumes = sorted_resumes[:5]

    # Store the top 5 resumes for this job description
    top_5_resumes_per_description[job_description] = top_5_resumes

# Print or process the top 5 resumes for each job description as needed
for job_description, top_5_resumes in top_5_resumes_per_description.items():
    print(f"Job Description: {job_description}")
    print("Top 5 Resumes:")
    for resume_details, cosine_similarity_score in top_5_resumes:
        filename = resume_details['filename']
        print(f"Resume: {filename}")
        print(f"Cosine Similarity: {cosine_similarity_score}")
    print("\n")


OutOfMemoryError: ignored